In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def split(array):
    dic={}
    for i in np.unique(array):
        dic.update({i:np.where(array==i)[0]})
    return dic

In [5]:
print(split(np.array([0,1,2])))
print(split(np.array([1,0,1,0,0,1,0])))
print(split(np.array([1,0,3,2,0,1,1])))

{0: array([0]), 1: array([1]), 2: array([2])}
{0: array([1, 3, 4, 6]), 1: array([0, 2, 5])}
{0: array([1, 4]), 1: array([0, 5, 6]), 2: array([3]), 3: array([2])}


In [6]:
import math
def entropy(array):
    b_list=[]
    for i in np.unique(array):
        p=len(np.where(array==i)[0])/len(array)
        b_list.append(p*math.log2(p))
    return -sum(b_list)

In [7]:
print(round(entropy(np.array([0,1,0,1,1,0])),4))
print(round(entropy(np.array([1,2])),4))
print(round(entropy(np.array([1,1])),4))
print(round(entropy(np.array([1,0,0,0,0,0,0,0,0,0,0])),4))
print(round(entropy(np.array([0,0,0])),4))
print(round(entropy(np.array([1,1,1,0,1,4,4,2,1])),4))

1.0
1.0
-0.0
0.4395
-0.0
1.6577


In [8]:
def IG(x,y):
    parent_entropy=entropy(y)
    split_dict=[split(x).get(k) for k in np.unique(x)]
    for i in split_dict:
        freq=len(x[[k for k in i]])/len(x)
        child_entropy=freq*entropy(y[[k for k in i]])
        parent_entropy=parent_entropy-child_entropy
    return parent_entropy

In [9]:
x=np.array([0,1,0,1,0,1])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))
x=np.array([0,0,1,1,2,2])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))
x=np.array([0,1,0,1,2,1])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))

0.4591
0.2516
0.9183


In [10]:
def make_tree(X,y,attribute):
    if y.shape[0]==1 or y.shape[0]==0:
        return y

    gains=[]
    if len(X.T)==1:
        gain=IG(X.T,y)
        if (gain<=1e-05):
            return y
        gains.append(gain)
    else:
        for x in X.T:
            gain=IG(x,y)

            if (gain<=1e-05):
                return y

            gains.append(gain)
    #print(gains)
    best_feature=np.argmax(gains)
    #print(best_feature)
    results={}
    
    subset_dict=split(X[:,best_feature])
    #print(subset_dict)
    for feature_value,train_example_indices in subset_dict.items():
        #print(train_example_indices)
        child_x_subset=np.delete(X[train_example_indices],best_feature,1)
        child_y_subset=y[train_example_indices]
        child_attribute=attribute[attribute != attribute[best_feature]]
        #print(child_x_subset)

        results["%s = %s" %(attribute[best_feature], feature_value)] = make_tree(child_x_subset, child_y_subset,child_attribute)

    return results

In [15]:
x=pd.read_csv("dataset/tennis.csv").iloc[:,1:-1].values
y=pd.read_csv("dataset/tennis.csv").iloc[:,-1].values
attribute=pd.read_csv("dataset/tennis.csv").iloc[:,1:-1].columns.values

In [16]:
print(attribute)
print(x)
print("label\n",y)
print("decision tree:\n",make_tree(x,y,attribute))

['outlook' 'temp' 'humidity' 'wind']
[['Sunny' 'Hot' 'High' 'Weak']
 ['Sunny' 'Hot' 'High' 'Strong']
 ['Overcast' 'Hot' 'High' 'Weak']
 ['Rain' 'Mild' 'High' 'Weak']
 ['Rain' 'Cool' 'Normal' 'Weak']
 ['Rain' 'Cool' 'Normal' 'Strong']
 ['Overcast' 'Cool' 'Normal' 'Strong']
 ['Sunny' 'Mild' 'High' 'Weak']
 ['Sunny' 'Cool' 'Normal' 'Weak']
 ['Rain' 'Mild' 'Normal' 'Weak']
 ['Sunny' 'Mild' 'Normal' 'Strong']
 ['Overcast' 'Mild' 'High' 'Strong']
 ['Overcast' 'Hot' 'Normal' 'Weak']
 ['Rain' 'Mild' 'High' 'Strong']]
label
 ['No' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes'
 'No']
decision tree:
 {'outlook = Overcast': array(['Yes', 'Yes', 'Yes', 'Yes'], dtype=object), 'outlook = Rain': {'wind = Strong': array(['No', 'No'], dtype=object), 'wind = Weak': array(['Yes', 'Yes', 'Yes'], dtype=object)}, 'outlook = Sunny': {'humidity = High': array(['No', 'No', 'No'], dtype=object), 'humidity = Normal': array(['Yes', 'Yes'], dtype=object)}}


In [17]:
def _traverse(x,d,attribute):
    if isinstance(d,np.ndarray):
        return d
    for key in d:
        name,value=key.split("=")
        feature_idx=attribute.tolist().index(name.strip())
        if x[feature_idx]==value.strip():
            return _traverse(x,d[key],attribute)

In [18]:
_traverse(np.array(['Rain','Mild','High','Weak']),make_tree(x,y,attribute),attribute)

array(['Yes', 'Yes', 'Yes'], dtype=object)

In [57]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

ModuleNotFoundError: No module named 'sklearn.externals.six'

In [38]:
golf_df=pd.read_csv("dataset/tennis.csv").iloc[:,1:]
golf_df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [39]:
one_hot_data = pd.get_dummies(golf_df[ ['outlook', 'temp', 'humidity', 'wind'] ])
one_hot_data

,outlook_Overcast,outlook_Rain,outlook_Sunny,temp_Cool,temp_Hot,temp_Mild,humidity_High,humidity_Normal,wind_Strong,wind_Weak
0,0,0,1,0,1,0,1,0,0,1
1,0,0,1,0,1,0,1,0,1,0
2,1,0,0,0,1,0,1,0,0,1
3,0,1,0,0,0,1,1,0,0,1
4,0,1,0,1,0,0,0,1,0,1
5,0,1,0,1,0,0,0,1,1,0
6,1,0,0,1,0,0,0,1,1,0
7,0,0,1,0,0,1,1,0,0,1
8,0,0,1,1,0,0,0,1,0,1
9,0,1,0,0,0,1,0,1,0,1


In [40]:
# The decision tree classifier.
clf = tree.DecisionTreeClassifier()
# Training the Decision Tree
clf_train = clf.fit(one_hot_data, golf_df['play'])

In [54]:
# Export/Print a decision tree in DOT format.
print(tree.export_graphviz(clf_train, None))

#Create Dot Data
dot_data = tree.export_graphviz(clf_train, out_file=None, feature_names=list(one_hot_data.columns.values), 
                                class_names=['not_play', 'play'], rounded=True, filled=True) 

#Create Graph from DOT data
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())

digraph Tree {
node [shape=box] ;
0 [label="X[0] <= 0.5\ngini = 0.459\nsamples = 14\nvalue = [5, 9]"] ;
1 [label="X[6] <= 0.5\ngini = 0.5\nsamples = 10\nvalue = [5, 5]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="X[8] <= 0.5\ngini = 0.32\nsamples = 5\nvalue = [1, 4]"] ;
1 -> 2 ;
3 [label="gini = 0.0\nsamples = 3\nvalue = [0, 3]"] ;
2 -> 3 ;
4 [label="X[5] <= 0.5\ngini = 0.5\nsamples = 2\nvalue = [1, 1]"] ;
2 -> 4 ;
5 [label="gini = 0.0\nsamples = 1\nvalue = [1, 0]"] ;
4 -> 5 ;
6 [label="gini = 0.0\nsamples = 1\nvalue = [0, 1]"] ;
4 -> 6 ;
7 [label="X[1] <= 0.5\ngini = 0.32\nsamples = 5\nvalue = [4, 1]"] ;
1 -> 7 ;
8 [label="gini = 0.0\nsamples = 3\nvalue = [3, 0]"] ;
7 -> 8 ;
9 [label="X[9] <= 0.5\ngini = 0.5\nsamples = 2\nvalue = [1, 1]"] ;
7 -> 9 ;
10 [label="gini = 0.0\nsamples = 1\nvalue = [1, 0]"] ;
9 -> 10 ;
11 [label="gini = 0.0\nsamples = 1\nvalue = [0, 1]"] ;
9 -> 11 ;
12 [label="gini = 0.0\nsamples = 4\nvalue = [0, 4]"] ;
0 -> 12 [labeldistance=

InvocationException: GraphViz's executables not found